In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import glob
import matplotlib.pyplot as plt
import seaborn as sns

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:

# Load all the EPC files clean and save as csvs

# needs pandas and geopandas

## Needs UPRN data
## uprn = pd.read_csv('./../Data/Ordinance_Survey/osopenuprn_202302.csv')
## uprn['UPRN'] = uprn['UPRN'].astype('float64')
import glob
import pandas as pd
import numpy as np 

def load_clean_epc(path_to_epc, path_save_cleaned, columns_keep, UPRN_data):
    #list_paths = glob.glob("../Data/Energy_Performance_Certificate/all-domestic-certificates/*")
    import glob
    import pandas as pd

    list_paths = glob.glob(path_to_epc)
    
    for i in range(len(list_paths)):
        path = list_paths[i] + '/certificates.csv'
        name = list_paths[i][-20:]
        epc = pd.read_csv(path, usecols=columns_keep) # columns = ['TOTAL_FLOOR_AREA','UPRN','LODGEMENT_DATE']
        # clean the uprn and add
        epc['UPRN'] = epc['UPRN'].astype('float64')
        epc2 = epc.merge(UPRN_data, on = 'UPRN',how = 'left')
        # clean duplicates
        epc2['LODGEMENT_DATE'] = pd.to_datetime(epc2['LODGEMENT_DATE'], format='%Y-%m-%d')
        epc2 = epc2.sort_values(by="LODGEMENT_DATE").drop_duplicates(subset=["UPRN"], keep="last")
        epc2 = epc2.loc[epc2.groupby('UPRN').LODGEMENT_DATE.idxmax()]
        # save to new folder
        epc2.to_csv(str(path_save_cleaned +  name + '.csv')) # path_save_cleaned = './../Data/Energy_Performance_Certificate/cerificates_processed/'
    
    return('done')

def load_compile_epc(path_to_cleaned):

    list_paths2 = glob.glob(path_to_cleaned) #path_to_cleaned = "./../Data/Energy_Performance_Certificate/cerificates_processed/*.csv"

    data = pd.DataFrame()
    for i in range(len(glob.glob(path_to_cleaned))):
        x = pd.read_csv(list_paths2[i])
        data = pd.concat([data,x])

    data = gpd.GeoDataFrame(data, geometry = gpd.points_from_xy(data.LONGITUDE,data.LATITUDE), crs = "EPSG:4326")

    return(data)



def extract_data_from_epc(path_to_certificates, columns, uprn_data_path, save_to_folder):
    # load paths
    uprn = pd.read_csv(uprn_data_path)
    list_paths = glob.glob(path_to_certificates)
    original_length = []
    cleaned_length = []
    name_la = []

    # foor loop to extract all the data
    for i in range(len(list_paths)):
        path = list_paths[i] + '/certificates.csv'
        #  define names
        name = list_paths[i][-20:]
        # 
        epc = pd.read_csv(path, usecols=columns)
        # write length
        original_length.append(len(epc))
        name_la.append(len(name))
        # correct the UPRN
        epc['UPRN'] = epc['UPRN'].astype('float64')
        # merge the data
        epc2 = epc.merge(uprn, left_on = ['UPRN','LOCAL_AUTHORITY'] , right_on = ['UPRN','LAD21CD'],how = 'left')
        # drop the duplicates
        epc2['LODGEMENT_DATE'] = pd.to_datetime(epc2['LODGEMENT_DATE'], format='%Y-%m-%d')
        epc2 = epc2.sort_values(by="LODGEMENT_DATE").drop_duplicates(subset=["UPRN"], keep="last")
        epc2 = epc2.loc[epc2.groupby('UPRN').LODGEMENT_DATE.idxmax()]
        # measure length
        cleaned_length.append(len(epc2))
        # write cleaned data
        epc2.to_csv(str(save_to_folder + name + '.csv'))
    
    pd.DataFrame(data = {'original': original_length, 'cleaned':cleaned_length, 'name':name_la}).to_csv(str(save_to_folder + 'lengths_assesment.csv'))
    


def load_all_cleaned_certificates(list_of_paths):
    list_of_paths = glob.glob(list_of_paths)
    data = pd.DataFrame()
    for i in range(len(list_of_paths)):
        x = pd.read_csv(list_of_paths[i])
        data = pd.concat([data,x])
    return(data) 



In [4]:
list_paths = glob.glob('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/certificates/*')

uprn = pd.read_csv('./../../Volumes/Extreme_SSD/WORK/MAV/UPRN/osopenuprn_202409.csv')



# ENERGY

In [3]:
for i in range(len(list_paths)):
    path = list_paths[i]
    name = list_paths[i][-10:]
    epc = pd.read_csv(path,skiprows=[1], usecols=['LODGEMENT_DATE','OSG_REFERENCE_NUMBER','MAINS_GAS_FLAG','ENERGY_CONSUMPTION_CURRENT',
                               'CURRENT_ENERGY_EFFICIENCY','CURRENT_ENERGY_RATING','ENERGY_CONSUMPTION_POTENTIAL'])
    epc = epc.rename(columns={'OSG_REFERENCE_NUMBER':'UPRN'})
    epc['UPRN'] = epc['UPRN'].astype('float64')
    epc2 = epc.merge(uprn, on = 'UPRN',how = 'left')
    epc2['LODGEMENT_DATE'] = pd.to_datetime(epc2['LODGEMENT_DATE'], format='%Y-%m-%d')
    epc2 = epc2.sort_values(by="LODGEMENT_DATE").drop_duplicates(subset=["UPRN"], keep="last")
    epc2 = epc2.loc[epc2.groupby('UPRN').LODGEMENT_DATE.idxmax()]
    epc2.to_csv(str('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_energy/' +  name ), index=False)


# HEAT

In [4]:
for i in range(len(list_paths)):
    path = list_paths[i]
    name = list_paths[i][-10:]
    epc = pd.read_csv(path,skiprows=[1], usecols=['LODGEMENT_DATE','OSG_REFERENCE_NUMBER','MAINHEAT_DESCRIPTION',
                                     'SECONDHEAT_DESCRIPTION','MAINS_GAS_FLAG','NUMBER_OPEN_FIREPLACES'])
    epc = epc.rename(columns={'OSG_REFERENCE_NUMBER':'UPRN'})
    epc['UPRN'] = epc['UPRN'].astype('float64')
    epc2 = epc.merge(uprn, on = 'UPRN',how = 'left')
    epc2['LODGEMENT_DATE'] = pd.to_datetime(epc2['LODGEMENT_DATE'], format='%Y-%m-%d')
    epc2 = epc2.sort_values(by="LODGEMENT_DATE").drop_duplicates(subset=["UPRN"], keep="last")
    epc2 = epc2.loc[epc2.groupby('UPRN').LODGEMENT_DATE.idxmax()]
    epc2.to_csv(str('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_heat/' +  name), index=False)


# LOSS

In [5]:
for i in range(len(list_paths)):
    path = list_paths[i]
    name = list_paths[i][-10:]
    epc = pd.read_csv(path, usecols=['LODGEMENT_DATE','OSG_REFERENCE_NUMBER','MAINS_GAS_FLAG','ROOF_DESCRIPTION',
                                     'EXTENSION_COUNT','MULTI_GLAZE_PROPORTION','PHOTO_SUPPLY'],skiprows=[1])
    epc = epc.rename(columns={'OSG_REFERENCE_NUMBER':'UPRN'})
    epc['UPRN'] = epc['UPRN'].astype('float64')
    epc2 = epc.merge(uprn, on = 'UPRN',how = 'left')
    epc2['LODGEMENT_DATE'] = pd.to_datetime(epc2['LODGEMENT_DATE'], format='%Y-%m-%d')
    epc2 = epc2.sort_values(by="LODGEMENT_DATE").drop_duplicates(subset=["UPRN"], keep="last")
    epc2 = epc2.loc[epc2.groupby('UPRN').LODGEMENT_DATE.idxmax()]
    epc2.to_csv(str('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_loss/' +  name ), index=False)



# HOUSE

In [6]:
for i in range(len(list_paths)):
    path = list_paths[i]
    name = list_paths[i][-10:]
    epc = pd.read_csv(path, usecols=['LODGEMENT_DATE','OSG_REFERENCE_NUMBER','MAINS_GAS_FLAG','TOTAL_FLOOR_AREA',
                                     'CONSTRUCTION_AGE_BAND','TENURE','BUILT_FORM','PROPERTY_TYPE'],skiprows=[1])
    epc = epc.rename(columns={'OSG_REFERENCE_NUMBER':'UPRN'})
    epc['UPRN'] = epc['UPRN'].astype('float64')
    epc2 = epc.merge(uprn, on = 'UPRN',how = 'left')
    epc2['LODGEMENT_DATE'] = pd.to_datetime(epc2['LODGEMENT_DATE'], format='%Y-%m-%d')
    epc2 = epc2.sort_values(by="LODGEMENT_DATE").drop_duplicates(subset=["UPRN"], keep="last")
    epc2 = epc2.loc[epc2.groupby('UPRN').LODGEMENT_DATE.idxmax()]
    epc2.to_csv(str('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_house/' +  name ), index=False)


# OTHER

In [11]:
for i in range(len(list_paths)):
    path = list_paths[i] 
    name = list_paths[i][-10:]
    epc = pd.read_csv(path,skiprows=[1], usecols=['LODGEMENT_DATE','OSG_REFERENCE_NUMBER','MAINS_GAS_FLAG', 
    'PROPERTY_TYPE','TRANSACTION_TYPE','FLOOR_LEVEL'])
    epc = epc.rename(columns={'OSG_REFERENCE_NUMBER':'UPRN'})
    epc2 = epc.merge(uprn, on = 'UPRN',how = 'left')
    epc2['LODGEMENT_DATE'] = pd.to_datetime(epc2['LODGEMENT_DATE'], format='%Y-%m-%d')
    epc2 = epc2.sort_values(by="LODGEMENT_DATE").drop_duplicates(subset=["UPRN"], keep="last")
    epc2 = epc2.loc[epc2.groupby('UPRN').LODGEMENT_DATE.idxmax()]
    epc2.to_csv(str('./../../Volumes/Extreme_SSD/WORK/EPC/scottish_EPC/cleaned_other/' +  name + '.csv'), index=False)